In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     80    8  463  9155.95  1834  9134.10  21.71  0.16  8896.94  1752   
10000    72   17  360  9130.44  1795  9108.77  21.54  0.14  8890.81  1742   
15000    69   26  257  9119.17  1779  9097.85  21.19  0.13  8887.29  1736   
20000    72   35  154  9112.62  1769  9091.58  20.91  0.12  8885.76  1734   
25000    69   44   51  9108.01  1763  9087.22  20.67  0.12  8883.87  1731   
...     ...  ...  ...      ...   ...      ...    ...   ...      ...   ...   
545000   73  961  112  9078.57  1719  9057.90  18.56  0.07  8882.88  1729   
550000   73  970    9  9078.51  1719  9057.82  18.56  0.07  8883.78  1731   
555000   73  978  473  9078.47  1719  9057.76  18.55  0.07  8882.62  1729   
560000   73  987  370  9078.43  1719  9057.72  18.55  0.07  8883.93  1730   
565000   73  996  267  9078.39  1719  9057.67  18.54  0.07  8883.26  1730   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    8874.80  22.02  0.12  8897.10  1751  0.43  0.46  0.41  0.32  0.23  
10000   8869.95  20.74  0.12  8890.63  1743  0.44  0.48  0.42  0.28  0.21  
15000   8866.96  20.22  0.11  8888.00  1737  0.44  0.47  0.43  0.29  0.21  
20000   8865.75  19.91  0.10  8884.86  1734  0.44  0.48  0.43  0.27  0.21  
25000   8864.23  19.54  0.10  8883.82  1731  0.46  0.47  0.44  0.28  0.21  
...         ...    ...   ...      ...   ...   ...   ...   ...   ...   ...  
545000  8864.73  18.09  0.06  8880.64  1726  0.46  0.49  0.45  0.26  0.21  
550000  8865.66  18.06  0.06  8880.64  1726  0.46  0.49  0.45  0.25  0.21  
555000  8864.54  18.03  0.06  8880.64  1726  0.46  0.49  0.44  0.26  0.21  
560000  8865.82  18.05  0.06  8880.64  1726  0.46  0.50  0.45  0.25  0.21  
565000  8865.14  18.06  0.06  8880.64  1726  0.46  0.49  0.44  0.25  0.21  

[113 rows x 20 columns]

0 R: 1.000 P: 0.121 user dialogue speech users human research systems linguistic domain utterance
   1 R: 0.231 P: 0.081 structure grammar rules semantic clause syntactic representation form structures type
     11 R: 0.073 P: 0.073 translation english source alignment phrase target translations parallel sentences pairs
     12 R: 0.077 P: 0.077 tree parsing dependency parser node rules algorithm grammar trees parse
   2 R: 0.191 P: 0.041 pos morphological tags annotation tag languages tagging dependency treebank parsing
     21 R: 0.077 P: 0.077 speech error errors character english chinese segmentation languages characters recognition
     22 R: 0.072 P: 0.072 training models embeddings neural learning network vector performance trained input
   3 R: 0.237 P: 0.057 verb semantic verbs noun syntactic lexical argument object role frame
     31 R: 0.073 P: 0.073 semantic sense terms wordnet lexical relations senses similarity domain term
     32 R: 0.108 P: 0.108 models probability algo

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     80    8  463  9155.95  1834  9134.10  21.71  0.16  8896.94  1752   
10000    72   17  360  9130.44  1795  9108.77  21.54  0.14  8890.81  1742   
15000    69   26  257  9119.17  1779  9097.85  21.19  0.13  8887.29  1736   
20000    72   35  154  9112.62  1769  9091.58  20.91  0.12  8885.76  1734   
25000    69   44   51  9108.01  1763  9087.22  20.67  0.12  8883.87  1731   
...     ...  ...  ...      ...   ...      ...    ...   ...      ...   ...   
545000   73  961  112  9078.57  1719  9057.90  18.56  0.07  8882.88  1729   
550000   73  970    9  9078.51  1719  9057.82  18.56  0.07  8883.78  1731   
555000   73  978  473  9078.47  1719  9057.76  18.55  0.07  8882.62  1729   
560000   73  987  370  9078.43  1719  9057.72  18.55  0.07  8883.93  1730   
565000   73  996  267  9078.39  1719  9057.67  18.54  0.07  8883.26  1730   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    8874.80  22.02  0.12  8897.10  1751  0.43  0.46  0.41  0.32  0.23  
10000   8869.95  20.74  0.12  8890.63  1743  0.44  0.48  0.42  0.28  0.21  
15000   8866.96  20.22  0.11  8888.00  1737  0.44  0.47  0.43  0.29  0.21  
20000   8865.75  19.91  0.10  8884.86  1734  0.44  0.48  0.43  0.27  0.21  
25000   8864.23  19.54  0.10  8883.82  1731  0.46  0.47  0.44  0.28  0.21  
...         ...    ...   ...      ...   ...   ...   ...   ...   ...   ...  
545000  8864.73  18.09  0.06  8880.64  1726  0.46  0.49  0.45  0.26  0.21  
550000  8865.66  18.06  0.06  8880.64  1726  0.46  0.49  0.45  0.25  0.21  
555000  8864.54  18.03  0.06  8880.64  1726  0.46  0.49  0.44  0.26  0.21  
560000  8865.82  18.05  0.06  8880.64  1726  0.46  0.50  0.45  0.25  0.21  
565000  8865.14  18.06  0.06  8880.64  1726  0.46  0.49  0.44  0.25  0.21  

[113 rows x 20 columns]

0 R: 1.000 P: 0.123 user dialogue users speech human research systems linguistic utterance utterances
   1 R: 0.230 P: 0.081 structure grammar rules semantic syntactic clause representation type form structures
     11 R: 0.073 P: 0.073 translation english source alignment phrase target translations sentences parallel pairs
     12 R: 0.076 P: 0.076 tree parsing dependency parser node algorithm rules grammar trees parse
   2 R: 0.188 P: 0.040 pos morphological tags tag annotation tagging languages dependency tagger treebank
     21 R: 0.077 P: 0.077 speech error segmentation errors character chinese english recognition training languages
     22 R: 0.071 P: 0.071 models training neural learning embeddings network performance trained vector input
   3 R: 0.239 P: 0.059 verb semantic verbs noun syntactic lexical frame argument role object
     31 R: 0.073 P: 0.073 semantic sense terms wordnet relations similarity lexical senses term concepts
     32 R: 0.107 P: 0.107 models probability a